In [18]:
!pip install --upgrade 'git+https://github.com/paulgavrikov/visualkeras'
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
import tensorflow.keras.optimizers
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
import os
from tensorflow.keras.regularizers import l2
import visualkeras
from PIL import ImageFont

import matplotlib.pyplot as plt
from matplotlib.pyplot import imread, imshow, imsave
import cv2
import numpy as np
import tensorflow
from sklearn.metrics import plot_confusion_matrix# This Python 3 environment comes with many helpful analytics libraries installed

  Cloning https://github.com/paulgavrikov/visualkeras to /tmp/pip-req-build-89z3uol7
  Running command git clone --filter=blob:none --quiet https://github.com/paulgavrikov/visualkeras /tmp/pip-req-build-89z3uol7
  Resolved https://github.com/paulgavrikov/visualkeras to commit d7111d3471173c0b16853a6ce5e4cc645498380a
  Preparing metadata (setup.py) ... done


In [2]:
def load(file_path):
    image = imread(file_path)
    return image
def display(image,title="Image"):
    plt.figure(figsize=[10,10])
    channels=len(image.shape)
    if channels<3:
        plt.imshow(image,cmap='Greys_r');
    else:
        plt.imshow(image);      
    plt.title(title);plt.axis("off"); 
    
def check_label(file_path):
    if os.path.getsize(file_path) == 0:
        return 0
    else :
        return 1
def load_data(directory):
    my_counter_0=0
    my_counter_1=0
    sequence=["x/train","x/test","x/valid"]
    train_data=[]
    test_data=[]
    valid_data=[]
    
    valid_labels=[]
    test_labels=[]    
    train_labels=[]
    for x in sequence:
        directory_search=directory+x[1:]+'/images'
        for filename in os.listdir(directory_search):
            f = os.path.join(directory_search, filename)
            if os.path.isfile(f):
                my_image=load(f)
                if len(my_image.shape)==1:
                    my_image = cv2.cvtColor(my_image, cv2.COLOR_GRAY2RGB) 
                
                resized = cv2.resize(my_image,(640,640), interpolation = cv2.INTER_AREA)
                if x == 'x/train' :
                    train_data.append(resized)
                elif x == 'x/test' :
                    test_data.append(resized)
                else:
                    valid_data.append(resized)
    
        directory_search=directory+x[1:]+'/labels'
        for filename in os.listdir(directory_search):
            f = os.path.join(directory_search, filename)
            if os.path.isfile(f):
                label=check_label(f)
                if label==0:
                    my_counter_0+=1
                elif label==1:
                    my_counter_1+=1
                if x == 'x/train' :
                    train_labels.append(label)
                elif x == 'x/test' :
                    test_labels.append(label)
                else:
                    valid_labels.append(label)
    print("Defected Pictures : ",my_counter_1)
    print("Non-Defected Pictures : ",my_counter_0)

    return train_data,test_data,valid_data,train_labels,test_labels,valid_labels

In [3]:
train_data,test_data,valid_data,train_labels,test_labels,valid_labels=load_data("/kaggle/input/finaldataset-autoinspect/")
np_train_data=np.array(train_data)
np_train_data = np_train_data.reshape((np_train_data.shape[0], 640,640, 3))
np_train_data = np_train_data.astype("float32") / 255.0
np_train_labels = np.asarray(train_labels).astype('float32').reshape((-1,1))

np_test_data=np.array(test_data)
np_test_data = np_test_data.reshape((np_test_data.shape[0], 640,640, 3))
np_test_data = np_test_data.astype("float32") / 255.0
np_test_labels = np.asarray(test_labels).astype('float32').reshape((-1,1))

np_valid_data=np.array(valid_data)
np_valid_data = np_valid_data.reshape((np_valid_data.shape[0], 640,640, 3))
np_valid_data = np_valid_data.astype("float32") / 255.0
np_valid_labels = np.asarray(valid_labels).astype('float32').reshape((-1,1))

Defected Pictures :  381
Non-Defected Pictures :  397


In [22]:
class AutoNet:    

    def build(imgRows, imgCols,numChannels,numClasses):
        # initialize the model
        model = Sequential()
        inputShape = (imgRows, imgCols, numChannels)
        
        model.add(Conv2D(32, 11, padding="same",input_shape=inputShape))
        model.add(Activation("relu"))


        model.add(Conv2D(64, 7, padding="same"))
        model.add(Activation("relu"))
        
        model.add(Conv2D(128, 5, padding="same"))
        model.add(Activation("relu"))


        
        model.add(Conv2D(32, 3, padding="same"))
        model.add(Activation("relu"))

        model.add(Conv2D(16, 3, padding="same"))
        model.add(Activation("relu"))
        model.add(keras.layers.MaxPooling2D(pool_size=(2, 2))) 
        model.add(BatchNormalization())

        model.add(Flatten())
        model.add(Activation("relu"))

        model.add(Dense(64))
        model.add(Activation("relu"))
        
        model.add(Dense(numClasses-1))
        model.add(Activation("sigmoid"))
        return model

In [23]:
model = AutoNet.build(numChannels=3, imgRows=640, imgCols=640,numClasses=2)
model.summary()
# font = ImageFont.truetype("arial.ttf", 32)  
visualkeras.layered_view(model, legend=True)  # font is optional!
visualkeras.layered_view(model,legend=True, to_file='output.png').show() # write and show

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_56 (Conv2D)           (None, 640, 640, 32)      11648     
_________________________________________________________________
activation_86 (Activation)   (None, 640, 640, 32)      0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 640, 640, 64)      100416    
_________________________________________________________________
activation_87 (Activation)   (None, 640, 640, 64)      0         
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 640, 640, 128)     204928    
_________________________________________________________________
activation_88 (Activation)   (None, 640, 640, 128)     0         
_________________________________________________________________
conv2d_59 (Conv2D)           (None, 640, 640, 32)     

Error: no "view" mailcap rules found for type "image/png"
/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening '/tmp/tmpqcj57mg8.PNG'


In [ ]:
model = AutoNet.build(numChannels=3, imgRows=640, imgCols=640,numClasses=2)
opt = keras.optimizers.Adam(learning_rate=0.02)
model.compile(loss="binary_crossentropy",optimizer=opt,metrics=["accuracy"])
model.fit(np_train_data, np_train_labels, batch_size=32, epochs=20,verbose=1,validation_data=(np_valid_data, np_valid_labels))

2023-01-06 18:26:49.049779: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-01-06 18:26:52.629216: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20
14/19 [=====================>........] - ETA: 2:47:37 - loss: 43.1838 - accuracy: 0.4732

In [ ]:
class estimator:
    _estimator_type = ''
    classes_=[]
    def __init__(self, model, classes):
        self.model = model
        self._estimator_type = 'classifier'
        self.classes_ = classes
    def predict(self, X):
        y_prob= self.model.predict(X)
        y_pred = y_prob.argmax(axis=1)
    return y_pred
class_names=[0,1]
classifier = estimator(model, class_names)

plot_confusion_matrix(estimator=classifier, X=np_test_data, y_true=np_test_labels)

In [ ]:
scores = model.evaluate(np_test_data, np_test_labels, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
model.save('AutoNet')